In [ ]:
import pandas as pd
import numpy as np
from sklearn import linear_model
import datetime
import time

In [ ]:
house_df = pd.read_csv('../data/properties_2016_backup.csv')

In [ ]:
# delete features with too many missing values
nan = house_df.isnull().sum()
nan = nan[nan < 2900000]
nan_feature = nan.index.tolist()
house_nan_df = house_df.loc[:,nan_feature]

In [ ]:
# delete categorical features
cate_feature = ['airconditioningtypeid','fips','heatingorsystemtypeid','pooltypeid7','propertycountylandusecode','propertylandusetypeid',
                'propertyzoningdesc','rawcensustractandblock','regionidcity','regionidcounty','regionidneighborhood','regionidzip',
                'censustractandblock']

for i in cate_feature:
    nan_feature.remove(i)

nan_cate_feature = nan_feature

house_nan_cate_df = house_nan_df.loc[:,nan_cate_feature]

In [ ]:
# delete highly correlated features

highcorr_feature = ['finishedsquarefeet12','finishedsquarefeet15', 'finishedsquarefeet50','taxvaluedollarcnt']

for i in highcorr_feature:
    nan_cate_feature.remove(i)

nan_cate_co_feature = nan_cate_feature

house_nan_co_cate_df = house_nan_cate_df.loc[:,nan_cate_co_feature]

# rename the dataframe, after deleting many features
house_new_df = house_nan_co_cate_df

In [ ]:
'''
counter = 0
for i in nan_cate_co_feature:
    counter = counter + 1
print(counter)
'''

In [ ]:
# house_new_df
# house_new_df.isnull().sum()
# house_new_df.shape`

In [ ]:
# fill the missing values with it's column's mean

house_new_df.fillna(house_new_df.mean(), inplace=True)

In [ ]:
house_new_df.isnull().sum()

In [ ]:
house_new_df_backup = house_new_df

cols_to_norm = ['bathroomcnt', 'bedroomcnt', 'buildingqualitytypeid', 'calculatedbathnbr', 'finishedfloor1squarefeet',
                'calculatedfinishedsquarefeet', 'fireplacecnt', 'fullbathcnt', 'garagecarcnt', 'garagetotalsqft', 'latitude', 'longitude',
                'lotsizesquarefeet', 'poolcnt', 'roomcnt', 'threequarterbathnbr', 'unitcnt', 'yearbuilt', 'numberofstories', 
                'structuretaxvaluedollarcnt', 'assessmentyear', 'landtaxvaluedollarcnt', 'taxamount']

house_new_df = house_new_df_backup
house_new_df[cols_to_norm] = house_new_df[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))


#house_new_df[cols_to_norm] = house_new_df[cols_to_norm].apply(lambda x: (x - x.mean()) / (x.std())

In [ ]:
house_new_df

In [ ]:
train_df = pd.read_csv('../data/train_2016_v2.csv')

In [ ]:
train_df

In [ ]:
join_df = train_df.merge(house_new_df, how='left', on='parcelid')

In [ ]:
join_df

In [ ]:
# training data set: prediction target

target = join_df.loc[:,'logerror']

In [ ]:
# training data set: attributes

train = join_df.iloc[:,2:]

# feature = join_df.iloc[:,2:]

In [ ]:
# poolcnt feature have problems during normalizations

del train['poolcnt']

In [ ]:
# convert datetime into float
# ToDo: normalize the datetime float

for num,str in enumerate(train.transactiondate):
        date_time = datetime.datetime.strptime(str,'%Y-%m-%d')
        str = time.mktime(date_time.timetuple())
        train.set_value(num,'transactiondate',str)
        
train['transactiondate'] = train['transactiondate'].astype(float)

In [ ]:
# train.isnull().sum()

In [ ]:
lr = linear_model.LinearRegression()
lr = lr.fit(train, target)

In [ ]:
# train.shape

In [ ]:
sample = pd.read_csv("../data/sample_submission.csv")

In [ ]:
sample1 = sample.ParcelId
sample1 = sample1.to_frame()
sample1.columns = ['parcelid']
sample1.describe()
df_test = sample1.merge(house_new_df, on='parcelid', how='left')

In [ ]:
del df_test['poolcnt']   # this need to be deleted before house_new_df is finished
del df_test['parcelid']

In [ ]:
# df_test.info()

In [ ]:
def pred(df_test):
    for num,str in enumerate(df_test.transactiondate):
        date_time = datetime.datetime.strptime(str,'%Y-%m-%d')
        str = time.mktime(date_time.timetuple())
        df_test.set_value(num,'transactiondate',str)
    df_test['transactiondate'] = df_test['transactiondate'].astype(float) 
    df_test = df_test.as_matrix()
    pred = lr.predict(df_test)
    return pred

In [ ]:
result = sample1.copy()
result.columns = ['ParcelId']

df_test1 = df_test.copy()
df_test1.insert(0,'transactiondate','2016-10-15')
pred1 = pred(df_test1)
pred1 = np.asarray(pred1)
result.insert(1,'201610',pred1)
del df_test1


df_test2 = df_test.copy()
df_test2.insert(0,'transactiondate','2016-11-15')
pred2 = pred(df_test2)
pred2 = np.asarray(pred2)
result.insert(2,'201611',pred2)
del df_test2

df_test3 = df_test.copy()
df_test3.insert(0,'transactiondate','2016-12-15')
pred3 = pred(df_test3)
pred3 = np.asarray(pred3)
result.insert(3,'201612',pred3)
del df_test3

df_test4 = df_test.copy()
df_test4.insert(0,'transactiondate','2017-10-15')
pred4 = pred(df_test4)
pred4 = np.asarray(pred4)
result.insert(4,'201710',pred4)
del df_test4

df_test5 = df_test.copy()
df_test5.insert(0,'transactiondate','2017-11-15')
pred5 = pred(df_test5)
pred5 = np.asarray(pred5)
result.insert(5,'201711',pred5)
del df_test5

df_test6 = df_test.copy()
df_test6.insert(0,'transactiondate','2017-12-15')
pred6 = pred(df_test6)
pred6 = np.asarray(pred6)
result.insert(6,'201712',pred6)
del df_test6


# result.columns = ['ParcelId', '201610']

result.columns = ['ParcelId', '201610', '201611', '201612', '201710','201711','201712']
result.to_csv('../predictions/sample1.csv',index=False,header=True)  # need to change filename per run